In [1]:
import anndata
import numpy as np
import sys
import torch
sys.path.append('../../../../')
import velovae as vv

In [2]:
dataset = 'Neuron_scNT'
adata = anndata.read_h5ad(f'data/{dataset}_pp.h5ad')
adata.obs['clusters'] = adata.obs['time'].astype(str)

In [ ]:
vv.preprocess(adata, n_gene=2000, min_shared_counts=20, keep_raw=True)

In [3]:
model_path_base = f'checkpoints/{dataset}'
figure_path_base = f'figures/{dataset}'
data_path = f'data/velovae/discrete/{dataset}'
gene_plot = ['Fosb','Rfx3','Cebpg','Homer1']

# Informative Time Prior

In [4]:
adata.obs['tprior'] = adata.obs['time'].to_numpy().astype(float)

# Discrete VeloVAE

In [6]:
figure_path = f'{figure_path_base}/DVAE'
model_path = f'{model_path_base}/DVAE'

torch.manual_seed(2022)
np.random.seed(2022)
dvae = vv.DVAE(adata, 
               tmax=20, 
               dim_z=5, 
               device='cuda:0',
               init_method='tprior',
               init_key='tprior',
               tprior='tprior',
               init_ton_zero=False)

dvae.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

dvae.save_model(model_path, 'encoder', 'decoder')
dvae.save_anndata(adata, 'dvae', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Initialization using prior time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
Library scale (U): Max=2.92, Min=0.48, Mean=1.03
Library scale (S): Max=2.50, Min=0.59, Mean=1.04
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 452.       *********
*********                      Stage  2                       *********


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.038
Average Set Size: 131
*********       Stage 2: Early Stop Triggered at epoch 485.       *********
*********              Finished. Total Time =   0 h :  2 m : 19 s             *********
Final: Train ELBO = -2435.957,           Test ELBO = -3321.507
       Training MSE = 1.987, Test MSE = 3.189


# Discrete Full VB

In [7]:
figure_path = f'{figure_path_base}/DFullVB'
model_path = f'{model_path_base}/DFullVB'

torch.manual_seed(2022)
np.random.seed(2022)
full_vb = vv.DVAEFullVB(adata, 
                        tmax=20, 
                        dim_z=5, 
                        device='cuda:0',
                        init_method='tprior',
                        init_key='tprior',
                        tprior='tprior',
                        init_ton_zero=False)

full_vb.train(adata, plot=False, gene_plot=gene_plot, figure_path=figure_path)

full_vb.save_model(model_path, 'encoder', 'decoder')
full_vb.save_anndata(adata, 'dfullvb', data_path, file_name=f'{dataset}.h5ad')

Detecting zero scaling factors: 0, 0
Initialization using prior time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Reinitialize the regular ODE parameters based on estimated global latent time.


  0%|          | 0/2000 [00:00<?, ?it/s]

Gaussian Prior.
Using informative time prior.
Library scale (U): Max=2.92, Min=0.48, Mean=1.03
Library scale (S): Max=2.50, Min=0.59, Mean=1.04
--------------------------- Train a VeloVAE ---------------------------
*********        Creating Training/Validation Datasets        *********
*********                      Finished.                      *********
*********                 Creating optimizers                 *********
*********                      Finished.                      *********
*********                    Start training                   *********
*********                      Stage  1                       *********
Total Number of Iterations Per Epoch: 17, test iteration: 32
*********       Stage 1: Early Stop Triggered at epoch 405.       *********
*********                      Stage  2                       *********


  0%|          | 0/3066 [00:00<?, ?it/s]

Percentage of Invalid Sets: 0.041
Average Set Size: 125
*********       Stage 2: Early Stop Triggered at epoch 671.       *********
*********              Finished. Total Time =   0 h :  3 m : 40 s             *********
Final: Train ELBO = -2569.436,           Test ELBO = -3506.016
       Training MSE = 3.618, Test MSE = 4.504


# Evaluation

In [ ]:
cluster_edges = [('0' , '15'), 
                 ('15', '30'), 
                 ('30', '60'), 
                 ('60', '120')]
vv.post_analysis(adata,
                 'eval',
                 ['Discrete VeloVAE','Discrete FullVB'],
                 ['dvae','dfullvb'],
                 compute_metrics=True,
                 genes=gene_plot,
                 grid_size=(1,4),
                 plot_type=['all'],
                 save_path=data_path,
                 cluster_edges=cluster_edges)